# QGrep Algorithm

In [26]:
# Import functions from grover_pattern.py and str_utils.py
import grover_pattern as gp
import str_utils as su
import numpy as np

## QGREP Wrapper Function

In [27]:
def run_qgrep(filename,target_str,threshold=0.8,iterations=12) -> None:
    """QGrep Front-End Wrapper Function
    
    :param filename: String for .txt file name (str)
    :param target_str: Target str to search (str)
    :param threshold: Probability threshold (float)
    :param iterations: Number of Qiskit circuit iterations (int)
    """
    
    # Get file data
    file = filename

    # Convert files to array of strings based on line numbers
    lines = su.get_lines_from_file(file)

    # Convert file and target to ints
    target = su.encode_str(target_str)
    encoding = su.strings2ints(lines)

    # Get inputs for oracle creation
    num_qubits = np.ceil(np.log2(encoding.max() + 1)).astype(int)
    num_iter = iterations
    master_indices = []

    for i, target_int in enumerate(target):
        # Create oracle
        qc = gp.grover_circuit(num_qubits, target_int, num_iter)

        # Simulate results
        probabilities = gp.match_ints(encoding.flatten(), qc)

        # Get indices of candidates
        indices = np.argwhere(probabilities >= threshold*max(probabilities))[:,0].tolist()

        # Compare indices and see if sequential -- potential solutions
        indices_to_remove = []
        if i == 0:
            master_indices = indices
        else:
            for j in master_indices:
                if j+i not in indices:
                    indices_to_remove.append(j)
        
        # Remove all failed comparisons
        for j in indices_to_remove:
            master_indices.remove(j)

    # Turn simulated data into user-legible output
    line_numbers = np.floor(np.asarray(master_indices) / encoding.shape[1]).astype(int)

    # Print all lines to user
    for i in line_numbers:
        print(f"({i})" + lines[i])

    print(f"{len(master_indices)} total lines found containing the string \"{target_str}\". Results printed above.\n")

## Experimental Results

### "Affability"

In [32]:
%%time
run_qgrep("prideandprejudice.txt","affability") 

(2627)      in a person of rank—such affability and condescension, as he had

(2634)      never seen anything but affability in her. She had always spoken

(5901)      not say you will be delighted with her. She is all affability and

(5992)      knowledge of her affability, that it would happen. But who could

4 total lines found containing the string "affability". Results printed above.

CPU times: user 34.3 s, sys: 46.1 ms, total: 34.3 s
Wall time: 31.9 s


### "Better"

In [33]:
%%time
run_qgrep("prideandprejudice.txt","better")

(225)      send them by themselves, which perhaps will be still better, for

(253)      “I desire you will do no such thing. Lizzy is not a bit better

(493)      better dance.”

(518)      who are slighted by other men. You had better return to your

(629)      still better, and say nothing of the bad—belongs to you alone.

(743)      “Yes; but he seemed to like his second better.”

(837)      sisters not worth speaking to, a wish of being better acquainted

(864)      In nine cases out of ten, a woman had better show _more_ affection

(904)      they both like Vingt-un better than Commerce; but with respect to

(916)      vexation; and it is better to know as little as possible of the

(1138)      their sisters’, and when nothing better offered, a walk to

(1228)      “No, my dear, you had better go on horseback, because it seems

(1266)      will not hear of my returning home till I am better. They insist also

(1326)      something better than politeness; there was good humour and


### "Darcy"

In [34]:
%%time
run_qgrep("prideandprejudice.txt","Darcy")

(455)      Hurst, merely looked the gentleman; but his friend Mr. Darcy soon

(474)      and his friend! Mr. Darcy danced only once with Mrs. Hurst and

(487)      Darcy had been standing near enough for her to overhear a

(491)      “Come, Darcy,” said he, “I must have you dance. I hate to see you

(507)      Mr. Darcy, looking at the eldest Miss Bennet.

(522)      Mr. Bingley followed his advice. Mr. Darcy walked off; and

(576)      spirit and some exaggeration, the shocking rudeness of Mr. Darcy.

(676)      Between him and Darcy there was a very steady friendship, in

(678)      Darcy by the easiness, openness, and ductility of his temper,

(681)      strength of Darcy’s regard Bingley had the firmest reliance, and

(682)      of his judgment the highest opinion. In understanding, Darcy was

(683)      the superior. Bingley was by no means deficient, but Darcy was

(687)      sure of being liked wherever he appeared, Darcy was continually

(696)      Darcy, on the contrary, had s

## Ground Truth Comparison

In [31]:
file = open("prideandprejudice.txt", "r")
data = file.read()

for target_string in ["affability","better","Darcy"]:
    occurrences = data.count(target_string)
    print(f'Number of occurrences of the word "{target_string}":', occurrences)

Number of occurrences of the word "affability": 4
Number of occurrences of the word "better": 92
Number of occurrences of the word "Darcy": 417
